# Setup

## Dependencies

In [ ]:
import pandas as pd
import os.path
from os import path
import requests
import json

## Files

Filepaths for relevant files (full dataset and trimmed dataset):

In [ ]:
hla_save_name = 'hla_save_dec19.xlsx'
hla_save_trimmed_name = 'hla_save_dec19_Bmm_only.tabular'

## Functions


Function for quickly obtaining column names:

In [ ]:
def get_cols(df, substring):
    return list(df.columns[df.columns.str.contains(substring)])

# HLA-B Leader 

## HLA SAVE

### Filtering
If the HLA SAVE file has not been trimmed to have only HLA-B single mismatches, then the following chunk runs code to do so. Otherwise, it reads from the trimmed file.

In [ ]:
if not path.exists(hla_save_trimmed_name):
    hla_save = pd.read_excel(hla_save_name)
    hla_save_b = hla_save.dropna(subset=['B_RES_MG']) # Drop rows without B locus match grades
    hla_save_b = hla_save_b[hla_save_b.B_RES_MG.str.contains('mm')] # Filter to rows with mismatches
    hla_save_b.to_csv(hla_save_trimmed_name)
else:
    hla_save_b = pd.read_csv(hla_save_trimmed_name)
hla_save_b.head(10)

## Columns

Let's examine what columns are available regarding the HLA-B locus:

In [ ]:
hla_save_b[get_cols(hla_save_b, 'B_')]

# REST Endpoints
Let's go through each REST call useful for leader annotation and process a row from our HLA dataset as inputs.

## HLA-B Leader Allotype Classification

### Row
This is the example row for the function set up. Feel free to change the indexing to a different row:

In [ ]:
row = hla_save_b.iloc[1]
row

### Parameters

The parameter for this first GET endpoint is simply a B allele. For example:

In [ ]:
param = row['R_B_TYP1']

### REST Call

This function calls the local server to classify the leader type of an HLA-B allele.

In [ ]:
def classify_leader_allotype(hla_b):
    if 'B*' not in hla_b:
        hla_b = 'B*' + hla_b
    url = "http://localhost:5010/b-leader-classification/allotype/" + hla_b
    headers = {'content-type': 'application/json'}
    try:
        response = requests.get(url,
                                headers=headers)
        results = json.loads(response.content)
        return results
    except Exception as e:
        raise e
classify_leader_allotype(param)

## HLA-B Leader Genotype Classification

This GET endpoint is essentially the same endpoint as the previous one but doubled. A genotype (containing two HLA-B allotypes) are leader classified in this endpoint.

### Parameter

Input for this endpoint is two HLA-B allotypes joined by a '+':

In [ ]:
def format_genotype_input(row, prefix):
    return '+'.join(['B*' + hla_b for hla_b in row[get_cols(hla_save_b, prefix + '_B_TYP')]])
params = format_genotype_input(row, 'R')
params

### REST Call
Note that this is a POST endpoint, rather than a GET endpoint. The leader genotype is classified here:

In [ ]:
def classify_leader_genotype(hla_b_genotype):
    print(hla_b_genotype)
    url = "http://localhost:5010/b-leader-classification/genotype/" + hla_b_genotype
    headers = {'content-type': 'application/json'}
    try:
        response = requests.get(url,
                                headers=headers)
        results = json.loads(response.content)
        return results
    except Exception as e:
        raise e
classify_leader_genotype(params)

## HLA-B Leader Matching
This endpoint classifies the leader types of the genotype of a patient and the genotype(s) of donor(s). Matching statuses and ranking information is included here as well:

### Parameter
The POST input parameter for this endpoint is formatted as follows:

In [ ]:
def format_matching_input(row):
    param = ('{"hla-b_genotype_patient":{"id": "%s", "genotype" : "%s"},'
              '"hla-b_genotype_donors":[{"id": "%s", "genotype" : "%s"}]}' %
             ('recipient id', format_genotype_input(row, 'R'),
              'donor id', format_genotype_input(row, 'D')))
    return param
params = format_matching_input(row)
params

### REST Call
The POST REST call takes in the above parameters:

In [ ]:
def classify_leader_matching(params):
    url = "http://localhost:5010/b-leader-match/status"
    headers = {'content-type': 'application/json'}
    try:
        response = requests.post(
             url,
            headers=headers,
            data=params)
        results = json.loads(response.content)
        return results
    except Exception as e:
        raise e
classify_leader_matching(format_matching_input(row))

# Iterating through the dataset
With all these functions set up, we can iterate through a list of recipient-donor typings to get our leader match statuses.

In [ ]:
for index, row in hla_save_b.iterrows():

    result = classify_leader_matching(format_matching_input(row))
    if len(result):
        hla_save_b.loc[index, ''] = result[0]['leader_match_status']
hla_save_b

# Done!
This is the end of the demonstration. Thank you! Please use this notebook and any of the above chunks for your own use.